# 💸 **FineNces - O sistema de Controle Financeiro**

## ✒️Autores - Grupo 2

- [Alessandra Cruz](https://github.com/alessandracruz)
- [Álex Buracosky](https://github.com/aburacosk)
- [Diana Osorio](https://github.com/diana468)
- [Diogo Moura](https://github.com/)
- [Felipe Zanardo](https://github.com/FelipeBZanardo)
- [Thiago Silva](https://github.com/)

## 📁 Repositório GitHub

- [Sistema_Controle_Financeiro](https://github.com/FelipeBZanardo/Sistema_Controle_Financeiro)

## 💸 Projeto Final | Sistema de Controle Financeiro

Deverá ser desenvolvido um sistema para controle financeiro que receba as movimentações e as armazena em um arquivo csv ou json.

O sistema deverá ser capaz de realizar as seguintes operações:

- **Criar** novos registros e identificar a data que o registro foi feito, qual tipo de movimentação, valor.

  - Os tipos podem ser:
    - Receita: o valor deve ser tratado como numérico e armazenado normalmente.
    - Despesas: o valor deve ser recebido como positivo, mas armazenado como negativo
    - Investimento: deve ter uma informação a mais de 'Montante', em que será calculado quanto o dinheiro rendeu desde o dia que foi investido.
    Para essa finalidade utilize a seguinte formula: $M = C * (1 + i)^t$ ([Saiba mais](https://matematicafinanceira.org/juros-compostos/)), considere tudo em dias.
- **Ler** registros: Deverá ser possível consultar os registros por data, tipo ou valor.
- **Atualizar** registros: No caso de atualização, pode-se atualizar o valor, o tipo e a data deverá ser a de atualização do registro.
- **Deletar**: Deverá ser possível deletar o registro (caso necessário, considere o indice do elemento como ID)

Outras funcionalidades:
- Crie uma função ```atualiza_rendimento``` que atualize os valores de rendimento sempre que chamada.
- Crie uma função ```exportar_relatorio```, que seja possível exportar um relatorio final em csv ou json.
- Crie pelo menos uma função de agrupamento, que seja capaz de mostrar o total de valor baseado em alguma informação (mes, tipo...)
- Crie valores separados para identificar a data (dia, mes, ano)

---

👩‍💻 **O que vai ser avaliado**:

- Se as funções e operações cuprem o seu objetivo
- Reprodutibilidade do código: vou executar!

👉🏻 **Envio do projeto**:
- Via LMS **individualmente.** <br>
  Apesar de ser em grupo, cada um de vocês precisa submeter o projeto.
- Formato: arquivo .py ou .ipynb.
- 📅 19/08, até as 23h59.

⚠️ **Atenção**:
- Não utilize a biblioteca pandas para resolução desse exercício


### Bibiotecas:

In [93]:
from datetime import datetime
from collections import namedtuple
import csv
from functools import reduce

### Modelos

In [94]:
Registro = namedtuple('Registro', ["dia", "mes", "ano", "tipo_movimentacao", "valor", "montante"])
Agrupamento = namedtuple('Agrupamento', ["tipo", "mês", "ano", "data", "valor"])
registros: dict[int: Registro] = {} #repositório em memória

### Utilitários

In [95]:
tipos_movimentacao: list[str] = ['receita', 'despesa', 'investimento']
tipos_consultas: list[str] = ['todos', 'data', 'tipo', 'valor']
tipos_alteracao: list[str] = ['tipo', 'valor']
tipos_agrupamento: list[str] = ['tipo', 'mês', 'ano', 'data']
nome_arquivo: str = 'banco_dados.csv'
opcoes_menu:list[str] = ['Criar Registro', 'Ler Registros', 'Atualizar Registro', 'Deletar Registro', 'Atualizar Rendimento', 'Exportar Relatório', 'Agrupar Registros', 'Sair']

In [96]:
def receber_tipo(tipo:str, lista_tipos: list[str]) -> str:
    """Aguarda o usuário a digitar um tipo válido de acordo com a lista_tipos

    Args:
        tipo (str): descrição do tipo utilizado no input
        lista_tipos (list[str]): lista com os tipos que estão disponíveis para a escolha do usuário

    Raises:
        ValueError: Exceção lançada para tratamento da entrada do usuário caso digite um tipo inválido

    Returns:
        str: retorna o tipo válido em formato de string
    """
    descricao:str = reduce(lambda x,y: x + ', ' + y, map(str.capitalize, lista_tipos))
    while True:
        try:
            tipo_recebido:str = input(f"Digite o tipo de {tipo} ({descricao}): ").lower().strip()
            if tipo_recebido not in lista_tipos:
                raise ValueError
        except ValueError:
            print("\033[1;30;31mTipo inválido! Tente novamente.\033[0m")
        else:
            print(f"\033[1;30;32mTipo de {tipo} '{tipo_recebido}' recebido com sucesso!\033[0m")
            return tipo_recebido

### Repositório:

In [97]:
def atualizar_repositorio_em_memoria(nome_arquivo_csv: str) -> None:
    """Responsável por ler o arquivo.csv que contém os dados dos Registros salvos e atualizar o repositório em memória.\n
    O repositório em memória é um dicionário com id como chave e os registros como valor

    Args:
        nome_arquivo_csv (str): Nome do arquivo.csv em string (não esquecer a extensão .csv) 
    """
    try:
        with open(nome_arquivo_csv, 'r', encoding='utf-8') as file:
                    leitor = csv.reader(file, delimiter=',', lineterminator='\n')
                    tabela_lida: list[list[str]] = [linha for linha in leitor][1:]
                    for linha in tabela_lida:
                            ident:int = int(linha[0])
                            dia, mes, ano = map(int,linha[1].split('/'))
                            tipo: str = linha[2]
                            valor: float = float(linha[3])
                            montante: float | None = float(linha[4]) if linha[4] != '' else None
                            registros[ident] = Registro(dia, mes, ano, tipo, valor, montante)
    except FileNotFoundError:
           arquivo = open(nome_arquivo, 'w', encoding='utf-8')
           arquivo.close()

def atualizar_arquivo(nome_arquivo_csv: str) -> None:
    """"Responsável por atualizar o arquivo.csv que contém os dados dos Registros salvos a partir do repositório em memória.\n
    O repositório em memória é um dicionário com id como chave e os registros como valor.\n
    O arquivo é armazenada da seguinte forma:\n
    \t'ID,Data,Tipo,Valor,Montante'

    Args:
        nome_arquivo_csv (str): _description_
    """
    dados_a_escrever: list[list[str]] = [['ID', 'Data', 'Tipo', 'Valor', 'Montante']]
    for ident, registro in registros.items():
        data_formatada:str = f"{registro.dia:02}/{registro.mes:02}/{registro.ano}"
        dados_a_escrever.append([ident, data_formatada, registro.tipo_movimentacao, registro.valor, registro.montante])

    with open(nome_arquivo_csv, 'w', encoding='utf-8') as file:
        csv.writer(file, delimiter=',', lineterminator='\n').writerows(dados_a_escrever)

    print(f"\033[1;30;32mDados salvo no arquivo {nome_arquivo_csv} com sucesso!\033[0m")


### 1) Criar Registro:

### ID:

In [98]:
#ID de cada registro é criado automaticamente
def incrementar_id() -> int:
    """Gera automaticamente um ID baseado no incremento do maior ID já existente.
    Caso não haja nenhum ID, é retornado 1.

    Returns:
        int: Retorna um ID do tipo inteiro
    """
    return 1 + (max([ident for ident in registros.keys()]) if len(registros) != 0 else 0)

### Data:

In [99]:
def receber_data() -> datetime:
    """Recebe uma data válida em formato 'DD/MM/YYYY' digitada pelo usuário\n
    Não admite:
    - Data fora do formato especificado
    - Datas inexistentes: exemplo: 30/02/2024 

    Returns:
        datetime: Retorna a data válida digitada pelo usuário em formato datetime
    """
    while True:
        data:str = input("Digite a data no formato 'DD/MM/YYYY': ")
        try:
            data_formatada: datetime = datetime.strptime(data, "%d/%m/%Y").date()
        except ValueError:
            print('\033[1;30;31mEssa data não existe ou está com formato inválido. Tente novamente!\033[0m')
        else:
            print(f"\033[1;30;32mData {data} recebida com sucesso!\033[0m")
            return data_formatada
    

### Valor da Movimentação:

In [100]:
def receber_valor() -> float:
    """Recebe um valor monetário válido digitado pelo usuário.\n
    Formato adequado: 00.00.\n
    Não admite:
    - Caracteres diferentes de números
    - Valores negativos
    - Uso de vírgula para separar real dos centavos

    Raises:
        ValueError: Lança exceção caso for fornecido valores não numéricos, valores negativos ou vírgula para separar real dos centavos.

    Returns:
        float: Retorna o valor monetário do tipo float digitado pelo usuário já no formato adequado
    """
    while True:
        try:
            valor:float = float(input("Digite o valor da movimentação: R$"))
            if valor <= 0: 
                raise ValueError
        except ValueError:
            print("\033[1;30;31mTente novamente! Digite um valor monetário válido (Use ponto para separar os centavos).\033[0m")
        else:
            print(f"\033[1;30;32mValor R${valor:.2f} recebido com sucesso!\033[0m")
            return valor

### Montante: 

In [101]:
def calcular_montante(valor: float, data_investimento: datetime) -> float:
    """Calcula o valor do montante gerado por juros compostos a partir da data do investimento até a data atual.\n
    Fórmula utilizada: M = C * (1 + i)^t
    - M: Montante (valor acrescido do rendimento conforme o dia e a taxa de juros compostos)
    - C: Capital (valor do investimento)
    - i: Taxa de juros (Para fins didáticos foi utilizado 0,1% a.d)
    - t: Tempo em que o valor está investido (Data do investimento - Data atual)\n

    **Para fins didáticos tanto a taxa de juros (i), quanto o tempo do investimento (t) devem ser em dias**

    Args:
        valor (float): Valor do Capital monetário investido com o tipo float
        data_investimento (datetime): Data do investimento no formato datetime

    Returns:
        float: Retorna o valor do Montante no formato float
    """
    data_atual: datetime = datetime.now().date()
    t: int = (data_atual - data_investimento).days
    i: float = 0.1 / 100 # 0,1% ao dia
    montante: float = valor * (1 + i)**t
    return montante

In [102]:
def criar_registro() -> Registro:
    """Cria um Registro no formato da namedTuple Registro(data, tipo_movimentacao, valor, montantante).\n
    - data, tipo_movimentacao, valor: são fornecidos pelo usuário
    - montante: é calculado se o tipo_movimentacao for 'investimento' ou 'None' para 'receita' e 'despesa'

    Returns:
        Registro: Retorna um registro de acordo com atributos escolhidos pelo usuário no formato namedTuple Registro(data, tipo_movimentacao, valor, montantante)
    """
    tipo_movimentacao: str = receber_tipo('movimentação', tipos_movimentacao)
    data: datetime = datetime.now().date()
    valor:float = -receber_valor() if tipo_movimentacao == 'despesa' else receber_valor()
    montante: float | None = calcular_montante(valor, data) if tipo_movimentacao == 'investimento' else None
    return Registro(data.day, data.month, data.year, tipo_movimentacao, valor, montante)

def salvar_registro(ident: int, registro: Registro) -> None:
    """Salva o registro no repositório, tanto em memória (dicionário que contém os registros) quanto no arquivo csv\n

    Args:
        id (int): é a chave do dicionário 'registros'
        registro (Registro): é o valor do dicionário 'registros' no formato namedTuple Registro(data, tipo_movimentacao, valor, montantante)
    """
    registros[ident] = registro
    atualizar_arquivo(nome_arquivo)
    


## 2) Ler Registros

In [103]:
def receber_valor_consulta(tipo_consulta: str) -> None | str | datetime | float :
    """Recebe um valor válido do usuário de acordo com o tipo de consulta solicitado.\n
    Tipos de consultas válidos: 
    - Todos (os registros)
    - Tipo (de movimentação)
    - Data 
    - Valor

    Args:
        tipo_consulta (str): Tipo de consulta válido (conforme descrição) em formato de string

    Returns:
        None | str | datetime | float: Retorna o valor de consulta válido digitado pelo usuário\n
        - None: caso o tipo_consulta for 'todos'
        - str: caso o tipo_consulta for 'tipo'
        - datetime: caso o tipo_consulta for 'data'
        - float: caso o tipo_consulta for 'valor'
    """
    if tipo_consulta == 'data':
        return receber_data()
    elif tipo_consulta == 'tipo':
        return receber_tipo('movimentação', tipos_movimentacao)
    elif tipo_consulta == 'valor':
        return receber_valor()
    elif tipo_consulta == 'todos':
        return None
    

def imprimir_registros(filtro: dict[Registro]) -> None:
    """Imprime no terminal os Registros em forma de tabela.\n
    Formato da tabela:
    - Título
    - Colunas:
        - ID
        - Data
        - Tipo Movimentação
        - Valor
        - Montante (apenas para investimentos)
        - Rendimento (apenas para investimentos)

    
    Args:
        filtro (dict[Registro]): Dicionário de Registros para impressão da tabela\nnamedTuple Registro: Registro(data, tipo_moviemntacao, valor, montante)
    """
    print('='*100)
    print(f"{'REGISTROS':^100}")
    print('='*100)
    print(f"{'ID':^4}   {'Data':^15}   {'Tipo Movimentação':^20}  {'Valor (R$)':^15}   {'Montante (R$)':^15}   {'Rendimento (R$)':^15}")
    print('='*100)
    for ident, registro in filtro.items():
        data:str = f"{registro.dia:0>2}/{registro.mes:0>2}/{registro.ano}"
        tipo:str = registro.tipo_movimentacao
        valor:float = registro.valor
        montante: str = f"{registro.montante:^15,.2f}" if registro.montante else ''
        rendimento: str = f"{(registro.montante - valor):^15,.2f}" if registro.montante else ''
        print(f"{ident:^4}   {data:^15}   {tipo.title():^20}  {valor:^15,.2f}   {montante}   {rendimento}")
        print('-'*100)

In [104]:
def ler_registros() -> None:
    """Gerencia:
    - O recebimento do tipo e do valor da consulta
    - Filtra os registros de acordo com a consulta
    - imprime os resultados no terminal em forma de tabela
    """
    tipo_consulta: str = receber_tipo('consulta', tipos_consultas)
    valor_consulta: None | str | datetime | float = receber_valor_consulta(tipo_consulta)

    filtros: dict[Registro] = {'todos': registros,
           'data': dict(filter(lambda x: datetime(x[1].ano, x[1].mes, x[1].dia).date() == valor_consulta, registros.items())),
           'tipo': dict(filter(lambda x: x[1].tipo_movimentacao == valor_consulta, registros.items())),
           'valor': dict(filter(lambda x: abs(x[1].valor) == valor_consulta, registros.items()))}
    
    filtro: dict[Registro] = filtros.get(tipo_consulta)
    imprimir_registros(filtro) 

### 3) Atualizar Registro

In [105]:
def receber_id(registros: dict[int:Registro]) -> int:
    """Recebe um id válido (que exista na base de dados) de um registro digitado pelo usuário.

    Args:
        registros (dict[int:Registro]): Dicionário que contém os Registros salvos na base de dados

    Raises:
        ValueError: Lança e trata uma execeção caso o usuário digite um id inválido

    Returns:
        int: Retorna um valor inteiro de um id válido digitado pelo usuário
    """
    ids = registros.keys()
    while True:
        try:
            ident: int = int(input("Digite o ID referente ao Registro: "))
            if ident not in ids:
                raise ValueError
        except ValueError:
            print("\033[1;30;31mTente novamente! Digite um ID válido\033[0m")
        else:
            print(f"\033[1;30;32mID {ident} recebido com sucesso!\033[0m")
            return ident

In [106]:
def atualizar_registro() -> tuple[int,Registro]:
    """Atualiza o registro de acordo com o id digitado pelo usuário.\n
    Para isso:
    - Imprime no terminal todos os registros
    - Aguarda um id válido escolhido pelo usuário para atualização do registro
    - Recebe do usuário o tipo de alteração no registro (pode alterar o tipo de movimentação ou o valor)
    - Recebe do usuário o novo valor a ser atualizado
    - Atualiza o repositório em memória e o arquivo de banco de dados

    Returns:
        tuple[int,Registro]: Retorna uma tupla com o id (inteiro) e o Registro atualizado no formato namedTuple Registro(data, tipo_movimentacao, valor, montantante)
    """
    imprimir_registros(registros)
    ident: int = receber_id(registros)

    data_atualizada: datetime = datetime.today().date()
    tipo_movimentacao: str = registros[ident].tipo_movimentacao
    valor:float = -registros[ident].valor if tipo_movimentacao == 'despesa' else registros[ident].valor

    tipo_alteracao:str = receber_tipo('alteração', tipos_alteracao)
    if tipo_alteracao == 'valor':
        valor:float = receber_valor()
    else:
        tipo_movimentacao:str = receber_tipo('movimentação', tipos_movimentacao)

    valor: float = -valor if tipo_movimentacao == 'despesa' else valor
    montante: float | None = calcular_montante(valor, data_atualizada) if tipo_movimentacao == 'investimento' else None
    return (ident, Registro(data_atualizada.day, data_atualizada.month, data_atualizada.year, tipo_movimentacao, valor, montante))
    

### 4) Deletar Registro

In [107]:
def confirmar_delete() -> bool:
    """Aguarda a confirmação do usuário para deleção do registro.\n
    's' para sim
    'n' para não

    Raises:
        ValueError: Lança exceção caso o usuário digite algo diferente de 's' ou 'n'

    Returns:
        bool: Retorna True caso o usuário digite 's' ou False caso o usuário digite 'n'
    """
    while True:
        try:
            confirmar: str = input("Tem certeza que deseja excluir o registro? [s/n]").strip().lower()
            if confirmar not in 'sn':
                raise ValueError
        except ValueError:
            print("\033[1;30;31mTente novamente! Digite 's' para sim ou 'n' para não\033[0m")
        else:
            return True if confirmar == 's' else False


In [108]:
def deletar_registro() -> None:
    """Deleta o registro de acordo com o id digitado pelo usuário.\n
    Para isso:
    - Imprime no terminal todos os registros
    - Aguarda um id válido escolhido pelo usuário para exclusão do registro
    """
    imprimir_registros(registros)
    id_delete: int = receber_id(registros)
    confirmar = confirmar_delete()
    if confirmar: 
        del registros[id_delete]
        print("\033[1;30;32mRegistro deletado com sucesso!\033[0m")
        atualizar_arquivo(nome_arquivo)

### 5) Atualizar Rendimento

In [109]:
def atualizar_rendimento(registros: dict[int:Registro]) -> None:
    """Atualiza o rendimento de todos os registros que são do tipo 'investimento'

    Args:
        registros (dict[int:Registro]): Dicionário que contém os Registros salvos na base de dados
    """
    
    for ident, registro in registros.items():
        data:datetime = datetime(registro.ano, registro.mes, registro.dia).date()
        valor:float = registro.valor
        tipo_movimentacao:str = registro.tipo_movimentacao
        montante:float = registro.montante
        if registro.tipo_movimentacao == 'investimento':
            montante:float = calcular_montante(valor, data)
        registros[ident] = Registro(data.day, data.month, data.year, tipo_movimentacao, valor, montante)

### 6) Exportar Relatório

In [110]:
def receber_nome_arquivo() -> str:
    """Recebe um nome válido para um arquivo.csv\n
    Aguarda o usuário digitar um nome sem espaços

    Raises:
        ValueError: Lança uma exceção caso o usuário digite um nome de arquivo com espaços

    Returns:
        str: Retorna o nome do arquivo.csv em formato de string
    """
    while True:
        try:
            arquivo:str = input("Digite o nome do arquivo.csv para exportar o relatório final: ").strip().lower()
            if len(arquivo.split(" ")) != 1:
                raise ValueError
        except ValueError:
            print("\033[1;30;31mNome inválido. Não digite espaços no nome.\033[0m")
        else:
            arquivo:str = arquivo if '.csv' in arquivo else (arquivo + '.csv')
            print(f"\033[1;30;32mArquivo '{arquivo}' recebido com sucesso!\033[0m")
            return arquivo

### 7) Agrupar Registros:

In [111]:
def agrupar(tipo_agrupamento: str, registros: dict[int: Registro]) -> list[tuple[str, float]]:
    """Agrupa os registros de acordo com o tipo de agrupamento e calcula o total dos valores.\n
    Tipos de agrupamentos válidos:
    - Tipo (de movimentação):
    - Mês
    - Ano
    - Data

    Args:
        tipo_agrupamento (str): tipo de agrupamento válido em formato de string
        registros (dict[int: Registro]): Dicionário que contém os Registros salvos na base de dados no formato namedTuple Registro(data, tipo_movimentacao, valor, montantante)

    Returns:
        list[tuple[str, float]]: Retorna uma lista de tuplas com os agrupamentos (em formato de string) e seus respectivos valores em formato de float
    """
    valores_agrupados: list[tuple[str, float]] = []

    registros_ordenados: dict[int: Registro] = dict(sorted(registros.items(), key=lambda x: (x[1].ano, x[1].mes, x[1].dia)))
    agrupamentos: list[Agrupamento] = list(map(lambda x: Agrupamento(x.tipo_movimentacao, f'{x.mes:02}/{x.ano}', x.ano, f'{x.dia:02}/{x.mes:02}/{x.ano}', x.valor), registros_ordenados.values()))

    valores_unicos = {getattr(valor, tipo_agrupamento): None for valor in agrupamentos}.keys() #não usado set para não interferir na ordenação

    for item in valores_unicos:
        filtro: filter = filter(lambda x: getattr(x, tipo_agrupamento) == item, agrupamentos)
        map_valor: map = map(lambda x: x.valor, filtro)
        soma: float = reduce(lambda x,y: x+y, map_valor, 0)
        valores_agrupados.append((item, soma))

    return valores_agrupados

In [112]:
def imprimir_agrupamento(tipo_agrupamento:str, agrupamentos: list[tuple[str, float]]) -> None:
    """Imprime no terminal uma tabela com o tipo de agrupamento e a soma dos valores relacionados

    Args:
        tipo_agrupamento (str): tipo de agrupamento válido em formato de string
        agrupamentos (list[tuple[str, float]]):  lista de tuplas com os agrupamentos (em formato de string) e seus respectivos valores em formato de float
    """
    print('='*35)
    print(f"{'AGRUPAMENTO':^35}")
    print('='*35)
    print(f"{tipo_agrupamento.upper():<20}   {'TOTAL (R$)':<15}")
    print('-'*35)
    for (chave, valor) in agrupamentos:
        print(f"{str(chave).capitalize():<20}   {valor:<15.2f}")

In [113]:
def agrupar_registros() -> None:
    """Agrupa os registros de acordo com um tipo escolhido pelo usuário e imprime no terminal
    """
    tipo_agrupamento: str = receber_tipo('agrupamento', tipos_agrupamento)
    agrupamento:list[tuple[str, float]] = agrupar(tipo_agrupamento, registros)
    imprimir_agrupamento(tipo_agrupamento, agrupamento)

### Menu

In [114]:
def menu_principal(opcoes: list[str]) -> None:
    """Menu principal para interação com o usuário para escolha de função do programa

    Args:
        opcoes (list[str]): Opções disponíveis para interação do usuário em formato de lista de string
    """
    while True:
        print("\n----------- Menu -----------")
        [print(f"{i+1}. {descricao}") for i,descricao in enumerate(opcoes)]
        print()

        opcao:str = input(f"Digite a opção de 1 a {len(opcoes)}: ")

        if opcao == '1':
            registro: Registro = criar_registro()
            print("\033[1;30;32mRegistro criado com sucesso!\033[0m")
            salvar_registro(incrementar_id(), registro)
        elif opcao == '2':
            ler_registros()
        elif opcao == '3':
            ident, registro = atualizar_registro()
            print("\033[1;30;32mRegistro atualizados com sucesso!\033[0m")
            salvar_registro(ident, registro)
        elif opcao == '4':
            deletar_registro()
        elif opcao == '5':
            atualizar_rendimento(registros)
            print("\033[1;30;32mRendimentos atualizados com sucesso!\033[0m")
            atualizar_arquivo(nome_arquivo)
        elif opcao == '6':
            arquivo: str = receber_nome_arquivo()
            atualizar_arquivo(arquivo)
            print(f"\033[1;30;32mRelatório exportado para o '{arquivo}' com sucesso!\033[0m")
        elif opcao == '7':
            agrupar_registros()
        elif opcao == '8':
            print("\033[1;30;32mFim do programa!\033[0m")
            break
        else:
            print(f"\033[1;30;31mTente novamente! Digite apenas números de 1 a {len(opcoes)}\033[0m")

### Programa Principal

In [115]:
atualizar_repositorio_em_memoria(nome_arquivo)

menu_principal(opcoes_menu)


----------- Menu -----------
1. Criar Registro
2. Ler Registros
3. Atualizar Registro
4. Deletar Registro
5. Atualizar Rendimento
6. Exportar Relatório
7. Agrupar Registros
8. Sair

Tipo de consulta 'todos' recebido com sucesso!
                                             REGISTROS                                              
 ID         Data          Tipo Movimentação      Valor (R$)       Montante (R$)    Rendimento (R$)
 3       12/08/2024          Investimento         1,000.00          1,006.02            6.02      
----------------------------------------------------------------------------------------------------
 5       14/08/2024            Despesa             -68.00           
----------------------------------------------------------------------------------------------------
 6       13/08/2024            Receita              85.99           
----------------------------------------------------------------------------------------------------
 8       12/08/2024          I